The city of Pittsburgh government publishes all the 311 requests (complaints about things like potholes and graffiti) it receives as open data on the [WPRDC data portal](https://data.wprdc.org).

To start playing with this data, we'll define a function that lets us extract information from the data portal's database by running SQL queries.

In [1]:
def query_resource(site,query):
    """Use the datastore_search_sql API endpoint to query a CKAN resource."""
    import ckanapi
    ckan = ckanapi.RemoteCKAN(site)
    response = ckan.action.datastore_search_sql(sql=query)
    return response['records']

Now let's specify the web site, the resource ID of the 311 data (which is just the end of the URL for the page where you can find the 311 data: [https://data.wprdc.org/dataset/311-data/resource/76fda9d0-69be-4dd5-8108-0de7907fc5a4](https://data.wprdc.org/dataset/311-data/resource/76fda9d0-69be-4dd5-8108-0de7907fc5a4)), and the category of 311 requests we're interested in:

In [2]:
site = "https://data.wprdc.org"
three_one_one_resource_id = "76fda9d0-69be-4dd5-8108-0de7907fc5a4"
category = "Potholes"

Now we can run the query, which is designed to sort by the CREATED_ON field (the date and time of the complaint) and pick just the most recent one:

In [3]:
query_resource(site,
query="SELECT * FROM \"{}\" WHERE \"REQUEST_TYPE\" = '{}' ORDER BY \"CREATED_ON\" DESC LIMIT 1".format(three_one_one_resource_id,category))

[{'STATUS': '0',
  'POLICE_ZONE': '5',
  'COUNCIL_DISTRICT': '7',
  'NEIGHBORHOOD': 'Bloomfield',
  'GEO_ACCURACY': 'EXACT',
  'REQUEST_ORIGIN': 'Website',
  'TRACT': '42003080200',
  '_full_text': "'+00':8 '-06':3 '-1':15 '-27':4 '-79.9518086':11 '0':17 '00':7,9 '10':6 '2':18 '2018':2 '264240':12 '3':14 '40.4614121':21 '42003080200':16 '5':19 '7':10 '8':22 'bloomfield':13 'dpw':23 'exact':26 'mainten':25 'pothol':20 'street':24 't13':5 'websit':1",
  'REQUEST_TYPE': 'Potholes',
  'PLI_DIVISION': '8',
  'Y': 40.4614121,
  'CREATED_ON': '2018-06-27T13:10:00',
  'PUBLIC_WORKS_DIVISION': '2',
  'REQUEST_ID': '264240',
  'DEPARTMENT': 'DPW - Street Maintenance',
  'X': -79.9518086,
  'WARD': '8',
  '_id': 252495,
  'FIRE_ZONE': '3-1'}]

The response gives us (among other details) the neighborhood where the pothole was reported, as well as the exact X and Y (longitude and latitude) coordinates of the pothole. (Some types of 311 requests do not have exact geocoordinates, in order to protect people's privacy.

One other intriguing field is STATUS, which has value of 0 (indicating that the request is new), 1 (indicating that the request has been closed), or 3 (indicating that the request is open).

In [9]:
def get_most_recent_311_request(category=None, neighborhood=None):
    """Get the most recent 311 complaint for the City of Pittsburgh based on category and neighborhood.
    
    Returns a string describing the request."""
    def query_resource(site,query):
        """Use the datastore_search_sql API endpoint to query a CKAN resource."""
        import ckanapi
        ckan = ckanapi.RemoteCKAN(site)
        response = ckan.action.datastore_search_sql(sql=query)
        return response['records']
        site = "https://data.wprdc.org"
    three_one_one_resource_id = "76fda9d0-69be-4dd5-8108-0de7907fc5a4"
    
    if category is None:
        if neighborhood is None:
            subset = query_resource(site,
    query="SELECT * FROM \"{}\" ORDER BY \"CREATED_ON\" DESC LIMIT 1".format(three_one_one_resource_id,category))
            record = subset[0]
            description = "The most recent 311 complaint for all of Pittsburgh was on {}, was about {}, and was for {}.".format(record['CREATED_ON'], record['REQUEST_TYPE'], record['NEIGHBORHOOD'])
        else:
            subset = query_resource(site,
    query="SELECT * FROM \"{}\" WHERE \"NEIGHBORHOOD\" = '{}' ORDER BY \"CREATED_ON\" DESC LIMIT 1".format(three_one_one_resource_id,neighborhood))
            record = subset[0]
            description = "The most recent 311 complaint for {} was on {} and was about {}.".format(record['NEIGHBORHOOD'], record['CREATED_ON'], record['REQUEST_TYPE'])
    else: # A 311 request category has been given
        if neighborhood is None:
            subset = query_resource(site,
    query="SELECT * FROM \"{}\" WHERE \"REQUEST_TYPE\" = '{}' ORDER BY \"CREATED_ON\" DESC LIMIT 1".format(three_one_one_resource_id,category))
            record = subset[0]
            description = "The most recent 311 {} complaint for all of Pittsburgh was on {} and was for {}.".format(record['REQUEST_TYPE'], record['CREATED_ON'], record['NEIGHBORHOOD'])
        else:
            subset = query_resource(site,
    query="SELECT * FROM \"{}\" WHERE \"REQUEST_TYPE\" = '{}' AND \"NEIGHBORHOOD\" = '{}' ORDER BY \"CREATED_ON\" DESC LIMIT 1".format(three_one_one_resource_id,category,neighborhood))
            record = subset[0]
            description = "The most recent 311 complaint for {} was on {} and was about {}.".format(record['NEIGHBORHOOD'], record['CREATED_ON'], record['REQUEST_TYPE'])
    
    return subset[0], description # This returns a dictionary object which has all the fields (like CREATED_ON) for the most recent request.


Let's use the above function to get the most recent 311 complaint about a dead animal.

In [10]:
dead_animal_record, latest_dead_animal_msg = get_most_recent_311_request(category="Dead Animal", neighborhood=None)

In [11]:
print(latest_dead_animal_msg)

The most recent 311 Dead Animal complaint for all of Pittsburgh was on 2018-06-27T07:23:00 and was for Garfield.


We could use the record and write a more specific kind of message:

In [12]:
output = "News flash! A new data animal was found in {}.".format(dead_animal_record['NEIGHBORHOOD'])
print(output)

News flash! A new data animal was found in Garfield.


But if we keep running the above code, we'll get multiple notifications about the same dead animal. To avoid this, let's keep track of the last request ID:

In [14]:
last_request_id = dead_animal_record['REQUEST_ID']
print(last_request_id)

264010


In [15]:
def check_for_new_dead_animal(previous_request_id, neighborhood=None):
    dead_animal_record, latest_dead_animal_msg = get_most_recent_311_request(category="Dead Animal", neighborhood=neighborhood)
    current_request_id = dead_animal_record['REQUEST_ID']
    if previous_request_id != current_request_id:
        output = "News flash! A new data animal was found in {}.".format(dead_animal_record['NEIGHBORHOOD'])
        print(output)
    else:
        output = None
    return current_request_id, output

To set up to check for the latest dead animal in Pittsburgh, first initial latest_request_id to None, so the first one it finds will be returned (the first time).

In [16]:
latest_request_id = None

Now you can keep running the command below, and (after the first run) it will only print a message if a new dead animal was found. The returned "output" value will contain the same message. Otherwise (if there's no new records), it will return an output value of None.

In [18]:
latest_request_id, output = check_for_new_dead_animal(latest_request_id)

## Using Twilio to place phone calls and send text messages

In [ ]:
import twilio

Below is a set of steps to set up your Twilio account. (Twilio's also got that own [walkthrough for getting started](https://www.twilio.com/docs/voice/quickstart/python).)
  
  

First, go to this link to sign up for an account:  
    [https://www.twilio.com/try-twilio](https://www.twilio.com/try-twilio)
    
It only takes a few minutes. You have to give them a phone number so they can text you a verification code. Once you input that, you should be up and running.



Go to this page:
    [https://www.twilio.com/console/voice/build/getting-started](https://www.twilio.com/console/voice/build/getting-started)
    
and click on the link that says "Show API Credentials".

In [24]:
TWILIO_ACCOUNT_SID="fillthisinwithyourown"
TWILIO_AUTH_TOKEN="googoosechewychewing"
if TWILIO_ACCOUNT_SID=="fillthisinwithyourown":
    from settings import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN

That page will also prompt you to choose a Twilio phone number. Copy and paste that number into the TWILIO_PHONE_NUMBER variable definition below.

In [31]:
TWILIO_PHONE_NUMBER = "+14125555555" # The Twilio phone number you picked.

Finally specify the number you want to call, by defining a NUMBER_TO_CALL in the cell below.

In [ ]:
NUMBER_TO_CALL = "+14125550000" # The number you want to place the call to. Change this to your own cell phone number.

Both phone numbers should start with a "+1" and then have ten more digits.

Now execute the block of code below. If you told Twilio to call your own phone, your phone should ring. Answer it!

When asked to enter a code, hit the "5" on the phone pad, and continue listening.

In [ ]:
# Download the helper library from https://www.twilio.com/docs/python/install
from twilio.rest import Client

# Your Account Sid and Auth Token can be found at twilio.com/console
client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

call = client.calls.create(
                        url='http://demo.twilio.com/docs/voice.xml',
                        to=NUMBER_TO_CALL,
                        from_=TWILIO_PHONE_NUMBER # formatted like this: '+15017122661'
                    )

print(call.sid)

Now let's use Twilio's sample code for sending text messages and make a simple function out of it.

In [44]:
def send_sms(text, target_number, your_twilio_number, account_sid, auth_token):
    from twilio.rest import Client
    client = Client(account_sid, auth_token)

    message = client.messages \
        .create(
             body=text,
             from_=your_twilio_number,
             to=target_number
         )

    print(message.sid)

Now we can send a message by calling that function with the message as the first argument.

In [46]:
send_sms("Where on Earth is the pizza guy?",NUMBER_TO_CALL,TWILIO_PHONE_NUMBER,TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN)

SM268ca79fdeeb41179f735d0c1059d290
